In [1]:
%pip install -U pip setuptools wheel
%pip install -U 'spacy[cuda-autodetect]'
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
2023-03-13 16:40:37.941832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
^C
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146

In [ ]:
!python3 -m spacy download en_core_web_lg -qq

2023-03-09 10:46:31.438359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 10:46:32.133322: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:46:32.133391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:46:32.133402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [1]:
import pandas as pd
sample_train = pd.read_csv(
    './train.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_test = pd.read_csv(
    './test.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_validation = pd.read_csv(
    './validation.csv', converters={'tag_set': eval, 'tag_list': eval})


In [2]:
import json
with open('selected_tags.json', 'r') as openfile:
    selected_tags = json.load(openfile)


In [3]:
from sklearn.model_selection import train_test_split
import spacy
from tqdm.notebook import tqdm
import numpy as np
from thinc.api import set_gpu_allocator, require_gpu
set_gpu_allocator("pytorch")
require_gpu(0)
nlp = spacy.load('en_core_web_lg')

def preprocess(texts):
    removal = ['ADV', 'PRON', 'CCONJ', 'PUNCT',
               'PART', 'DET', 'ADP', 'SPACE', 'NUM', 'SYM']
    tokens = []
    cleaned_texts = []
    print("preprocessing")
    for summary in tqdm(nlp.pipe(texts, disable=["tok2vec"]), total=len(texts)):
        question_tokens = []
        for token in summary:
            if token.pos_ not in removal and not token.is_stop and token.is_alpha:
                question_tokens.append(token.lemma_)
        cleaned_texts.append(" ".join(question_tokens))
    # question_tokens = [token.lemma_ for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    return cleaned_texts

def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    print("converting")
    for doc, labels in tqdm(nlp.pipe(data.values, as_tuples=True), total=len(data)):
        for tag in selected_tags:
            doc.cats[tag] = tag in labels
        db.add(doc)

    db.to_disk(outfile)



2023-03-17 11:39:00.485822: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 11:39:02.620787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 11:39:02.621221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 11:39:02.621394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [4]:
train_df = sample_train[~sample_train.text.isna()].loc[:]
train_df["text_processed"] = preprocess(train_df.text)
convert(train_df.loc[:, ["text_processed", "tag_list"]],
        'text_train_bow.spacy')
test_df = sample_test[~sample_test.text.isna()].loc[:]
test_df["text_processed"] = preprocess(test_df.text)
convert(test_df.loc[:, ["text_processed", "tag_list"]], 'text_test_bow.spacy')
validation_df = sample_validation[~sample_validation.text.isna()].loc[:]
validation_df["text_processed"] = preprocess(
    validation_df.text)
convert(validation_df.loc[:, ["text_processed", "tag_list"]],
        'text_validation_bow.spacy')


preprocessing


  0%|          | 0/105710 [00:00<?, ?it/s]

converting


  0%|          | 0/105710 [00:00<?, ?it/s]

preprocessing


...

  0%|          | 0/21648 [00:00<?, ?it/s]

preprocessing


  0%|          | 0/14149 [00:00<?, ?it/s]

converting


  0%|          | 0/14149 [00:00<?, ?it/s]

In [1]:
!python3 -m spacy train config_BOW.cfg --output ./output_BOW --paths.train ./text_train_bow.spacy --paths.dev ./text_test_bow.spacy --gpu-id 0

2023-03-17 12:18:13.716779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 12:18:15.191947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:18:15.192320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 12:18:15.192476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [13]:
!python3 -m spacy evaluate ./output_BOW/model-best/ ./text_validation_bow.spacy

2023-03-13 18:54:16.642354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 18:54:19.589942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 18:54:19.590303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 18:54:19.590453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [ ]:
texts = ["how to center a div", "list comprehension in python"]
model = spacy.load("./output_BOW/model-best")
for text in texts:
    doc = model(preprocess([text])[0])
    print(doc.cats,  "-",  text)


preprocessing


  0%|          | 0/1 [00:00<?, ?it/s]

{'java': 0.07408298552036285, 'iphone': 0.15504322946071625, '.net': 0.10662419348955154, 'c': 0.1426505446434021, 'php': 0.09644465893507004, 'css': 0.40599536895751953, 'python': 0.0642370954155922, 'ios': 0.1259569674730301, 'c++': 0.071302130818367, 'android': 0.07744339108467102, 'c#': 0.09655748307704926, 'html': 0.45438024401664734, 'node.js': 0.08022075146436691, 'javascript': 0.21241994202136993, 'objective-c': 0.05602111294865608, 'jquery': 0.17538514733314514} - how to center a div
preprocessing


  0%|          | 0/1 [00:00<?, ?it/s]

{'java': 0.05628538876771927, 'iphone': 0.06126730144023895, '.net': 0.06102268025279045, 'c': 0.09937193244695663, 'php': 0.034316014498472214, 'css': 0.09607839584350586, 'python': 0.9698299169540405, 'ios': 0.09190020710229874, 'c++': 0.0937778651714325, 'android': 0.04890865832567215, 'c#': 0.06937907636165619, 'html': 0.14873839914798737, 'node.js': 0.04805421084165573, 'javascript': 0.08540388941764832, 'objective-c': 0.039352498948574066, 'jquery': 0.0537249781191349} - list comprehension in python


In [5]:
train_df = sample_train[~sample_train.code.isna()].loc[:]
train_df["code_processed"] = preprocess(train_df.code)
convert(train_df.loc[:, ["code_processed", "tag_list"]],
        'code_train_bow.spacy')
test_df = sample_test[~sample_test.code.isna()].loc[:]
test_df["code_processed"] = preprocess(test_df.code)
convert(test_df.loc[:, ["code_processed", "tag_list"]], 'code_test_bow.spacy')
validation_df = sample_validation[~sample_validation.code.isna()].loc[:]
validation_df["code_processed"] = preprocess(
    validation_df.code)
convert(validation_df.loc[:, ["code_processed", "tag_list"]],
        'code_validation_bow.spacy')


preprocessing


  0%|          | 0/81619 [00:00<?, ?it/s]

converting


  0%|          | 0/81619 [00:00<?, ?it/s]

preprocessing


...

  0%|          | 0/16800 [00:00<?, ?it/s]

preprocessing


  0%|          | 0/10956 [00:00<?, ?it/s]

converting


  0%|          | 0/10956 [00:00<?, ?it/s]

In [2]:
!python3 -m spacy train config_BOW.cfg --output ./output_code_BOW --paths.train ./code_train_bow.spacy --paths.dev ./code_test_bow.spacy --gpu-id 0

2023-03-17 13:35:08.530678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 13:35:10.172390: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:35:10.172770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 13:35:10.172917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [1]:
!python3 -m spacy evaluate ./output_code_BOW/model-best/ ./text_validation_bow.spacy --gpu-id 0

2023-03-14 18:54:28.647573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 18:54:31.160458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 18:54:31.160817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 18:54:31.160976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on